In [1]:
from src.lexer.hulk_lexer import HulkLexer
from src.hulk_grammar.hulk_grammar import G
from src.parsing import LR1Parser


lexer = HulkLexer()
parser = LR1Parser(G)

In [2]:
from src.evaluation import evaluate_reverse_parse
from src.semantics.semantic_analysis_pipeline import semantic_analysis_pipeline


tokens, errors = lexer('''
    type A(x: A, y: G) {
        suma ( a : Number , b : B ) : Number {
            a + b ;
        }
        resta (g: Number, d : Number) : Number {
            g + d;
        }
        b : Number =89;
        c : C =884;
    }

    protocol G {
        suma ( c : Number , d : B ) : Number;
    }

    protocol H extends G {
        resta ( c : Number , d : B ) : Number;
    }

    type B inherits A {
        c : A =59;
        f ( d : Number , a : A ) : Number {
            c ;
        }
        z : Number =9598;
    }

    type C(a, b) {
        suma ( a : Number , b : B ) : Number {
            a + b ;
        }
    }

    type D(z) inherits A {
        suma ( a : Number , d : B ) : Number {
            d ;
        }
    }

    type E inherits A {
        suma ( a : A , b : B ) {
            a ;
        }
    }

    type F inherits B {
        f ( self : Number , a : A ) {
            a ;
        }
    }

    function s() {
        let a : A= print(A) , a = 4, a= 8 in print ( a) ;
    }

    type Point(x, y) {
        x = x;
        y = y;
    }

    type PolarPoint(phi, rho, z) inherits Point(z + phi, rho + z){
        x = phi;
        y = rho;
    }

    let a : A = b in print ( a) ;
    '''
)

derivation, operations = parser([t.token_type for t in tokens])
ast = evaluate_reverse_parse(derivation, operations, tokens)
ast, errors, context, scope = semantic_analysis_pipeline(ast)

In [7]:
def get_C_att_name(attribute_type):
    match attribute_type.name:
        case "Number":
            return "double"
        
    return attribute_type.name + "*"

code = ""

create_defs = {}
method_defs = {}

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        create_def = "Object* create" + type.name + " ("
        create_params = []

        current = type

        while current is not None:
            for att in current.attributes:
                create_params.append(current.name + "_" + att.name)
                create_def += "Object* " + create_params[-1] + ", "

            current = current.parent

        if len(create_params) > 0:
            create_def = create_def[:-2]
        
        create_def += ")"

        create_defs[type.name] = (create_def, create_params)

        code += create_def + ";\n"

        method_defs[type.name] = []

        for method in type.methods:
            method_name = "fun_" + type.name + "_" + method.name
            method_def = "Object* " + method_name + " (Object* self"
            
            method_id_params = {}
            
            for param_name in method.param_names:
                id_param = "p_" + param_name
                method_id_params[param_name] = id_param
                method_def += ", Object* " + id_param

            method_def += ")"
            method_defs[type.name].append((method_def, method_name, method_id_params, method))
            code += method_def + ";\n"
                
        code += "\n"

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        code += create_defs[type.name][0] + " {\n"
        code += "   Object* obj = createObject();\n"
        code += "   addAttribute(obj, \"type\",\"" + type.name + "\");\n"

        code += "\n"
        for param in create_defs[type.name][1]:
            code += "   addAttribute(obj, \"" + param + "\", " + param + ");\n"

        code += "\n"

        current = type

        while current is not None and current.name != "Object":
            if current.name in method_defs:
                for method in method_defs[current.name]:
                    code += "   addAttribute(obj, \"" + method[1] + "\", *" + method[1] + ");\n"

            current = current.parent

        code += "}\n\n"

for type in context.types.values():
    if type.name not in ["Number", "Bool", "String", "Object"]:
        if type.name in method_defs:
            for method_def, method_name, method_id_params, method in method_defs[type.name]:
                code += method_def + " {\n"

                code += str(method.node) + "\n"

                code += "}\n\n"
        

print(code)

Object* createA (Object* A_b, Object* A_c);
Object* fun_A_suma (Object* self, Object* p_a, Object* p_b);
Object* fun_A_resta (Object* self, Object* p_g, Object* p_d);

Object* createB (Object* B_c, Object* B_z, Object* A_b, Object* A_c);
Object* fun_B_f (Object* self, Object* p_d, Object* p_a);

Object* createC ();
Object* fun_C_suma (Object* self, Object* p_a, Object* p_b);

Object* createD (Object* A_b, Object* A_c);
Object* fun_D_suma (Object* self, Object* p_a, Object* p_d);

Object* createE (Object* A_b, Object* A_c);
Object* fun_E_suma (Object* self, Object* p_a, Object* p_b);

Object* createF (Object* B_c, Object* B_z, Object* A_b, Object* A_c);
Object* fun_F_f (Object* self, Object* p_self, Object* p_a);

Object* createPoint (Object* Point_x, Object* Point_y);

Object* createPolarPoint (Object* PolarPoint_x, Object* PolarPoint_y);

Object* createA (Object* A_b, Object* A_c) {
   Object* obj = createObject();
   addAttribute(obj, "type","A");

   addAttribute(obj, "A_b", A_b);
 